In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-190737
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-190737


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "projectcluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.01,0.1,1,10),
        '--max_iter': choice(50,100,200)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#env = Environment.get(workspace = ws, name = 'AzureML-Tutorial')
#env = Environment('user-managed-env')
#env.python.user_managed_dependencies = True

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory="./", ##Users/odl_user_188259/train.py
                      script='train.py',
                      arguments=None,
                      compute_target=cpu_cluster, #compute_config
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config = src,
    hyperparameter_sampling = ps,
    policy = policy, 
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs =  16,
    max_concurrent_runs = 4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

remote_run = exp.submit(config = hyperdrive_config)
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run = remote_run.get_best_run_by_primary_metric()
print(best_run.get_details())
remote_run.get_children_sorted_by_primary_metric(top=1)

{'runId': 'HD_82a79f18-d3f0-464b-89db-12fc9536687d_1', 'target': 'projectcluster', 'status': 'Completed', 'startTimeUtc': '2022-04-01T09:44:29.721051Z', 'endTimeUtc': '2022-04-01T09:47:15.238366Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '7333c288-578d-4ea3-989d-e630414831c7', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.1', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'projectcluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-env', 'version

[{'run_id': 'HD_82a79f18-d3f0-464b-89db-12fc9536687d_1',
  'hyperparameters': '{"--C": 0.1, "--max_iter": 100}',
  'best_primary_metric': 0.9088012139605463,
  'status': 'Completed'}]

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5, 
    compute_target=cpu_cluster)

In [11]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on projectcluster with default configuration
Running on remote compute: projectcluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ef990a0b-7f88-485b-a5e0-2508362553ab,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [12]:
# Retrieve and save your best automl model.

best_model = automl_run.get_best_child()

best_model.get_details()

{'runId': 'AutoML_ef990a0b-7f88-485b-a5e0-2508362553ab_30',
 'target': 'projectcluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-01T10:27:16.390756Z',
 'endTimeUtc': '2022-04-01T10:27:20.087054Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'projectcluster\',\'subscription_id\':\'610d6e37-4747-4a20-80eb-3aad70a55f43\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_ef990a0b-7f88-485b-a5e0-2508362553ab_30","experiment_name":"udacity-project","workspace_name":"quick-starts-ws-190737","subscription_id":"

In [13]:
best_model.get_metrics()

{'f1_score_micro': 0.9183915022761759,
 'precision_score_weighted': 0.9154350481159907,
 'accuracy': 0.9183915022761759,
 'average_precision_score_micro': 0.9817887585948102,
 'AUC_weighted': 0.9485785186527632,
 'matthews_correlation': 0.5739095049406739,
 'balanced_accuracy': 0.7749169666773186,
 'recall_score_micro': 0.9183915022761759,
 'norm_macro_recall': 0.5498339333546373,
 'precision_score_macro': 0.7996529848149739,
 'precision_score_micro': 0.9183915022761759,
 'f1_score_weighted': 0.9166734202853515,
 'recall_score_macro': 0.7749169666773186,
 'average_precision_score_macro': 0.8280526158023835,
 'recall_score_weighted': 0.9183915022761759,
 'f1_score_macro': 0.7863548634170818,
 'AUC_macro': 0.9485785186527632,
 'weighted_accuracy': 0.9540399412564996,
 'log_loss': 0.21774548436544272,
 'AUC_micro': 0.9810580108270912,
 'average_precision_score_weighted': 0.9562486496715966,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_ef990a0b-7f88-485b-a5e0-2508362553ab

In [14]:
best_model

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ef990a0b-7f88-485b-a5e0-2508362553ab_30,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
# {
#     "class_name": "LightGBMClassifier",
#     "module": "automl.client.core.common.model_wrappers",
#     "param_args": [],
#     "param_kwargs": {
#         "boosting_type": "goss",
#         "colsample_bytree": 0.7922222222222222,
#         "learning_rate": 0.09473736842105263,
#         "max_bin": 180,
#         "max_depth": 3,
#         "min_child_weight": 7,
#         "min_data_in_leaf": 0.04828103448275863,
#         "min_split_gain": 0.894736842105263,
#         "n_estimators": 100,
#         "num_leaves": 95,
#         "reg_alpha": 0.5789473684210527,
#         "reg_lambda": 0.5263157894736842,
#         "subsample": 1
#     },
#     "prepared_kwargs": {},
#     "spec_class": "sklearn"
# }

In [15]:
# Cluster cleanup
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

